# Setup

### Import necessary modules and do some basic setup.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.utils import class_weight

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers, regularizers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pickle
import pathlib
import hashlib
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dotenv
from dotenv import dotenv_values

# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_resnet import *
from utils.utils_plot import *
from utils.DNN_models import *

Num GPUs Available:  1


In [2]:
import yaml
conf = yaml.safe_load(open("config.yaml"))
PRECIP_XTRM = 0.99 # Percentile (threshold) for the extremes

In [3]:
i_shape = conf['i_shape']
o_shape = conf['o_shape']

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')
output_channels = conf['output_channels']
num_filters = conf['num_filters']
use_batchnorm = conf['use_batchnorm']
dropout = conf['dropout']
lr = conf['lr']

name_model = conf['model']
output_scaling = 1
output_crop = None


# load coordinates
lons_x = np.load('tmp/data/lons_y.npy')
lats_y = np.load('tmp/data/lats_y.npy')

# load precip
if PRECIP_XTRM == 0.95:
    pr_xtrm = np.load('tmp/data/pr_xtrm_95.npy')
elif PRECIP_XTRM == 0.99:
    pr_xtrm = np.load('tmp/data/pr_xtrm_99.npy')
#pr_xtrm = np.load('tmp/data/pr_xtrm_99.npy')
# create a time array
times = np.arange(np.datetime64('1979-01-01'), np.datetime64('2006-01-01')) #until validation period
times = pd.to_datetime(times)

X shape: [46, 56, 31]
y shape: [46, 56, 1]


In [4]:
# load the training and testing data
dg_train_X = np.array(xr.open_dataarray('tmp/data/dg_train_X.nc'))
dg_train_Y = np.array(xr.open_dataarray('tmp/data/dg_train_Y.nc'))

dg_valid_X = np.array(xr.open_dataarray('tmp/data/dg_valid_X.nc'))
dg_valid_Y = np.array(xr.open_dataarray('tmp/data/dg_valid_Y.nc'))


test_times = np.arange(np.datetime64('2016-01-01'), np.datetime64('2022-01-01')) #until validation period
test_times = pd.to_datetime(test_times)
dg_test_X = np.array(xr.open_dataarray('tmp/data/dg_test_X.nc'))
dg_test_Y = np.array(xr.open_dataarray('tmp/data/dg_test_Y.nc'))

# Open files for extremes
if PRECIP_XTRM == 0.95:
    dg_train_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_train_Y_xtrm0.95th.nc'))
    dg_valid_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_valid_Y_xtrm0.95th.nc')) 
    dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm0.95th.nc'))
elif PRECIP_XTRM == 0.99:
    dg_train_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_train_Y_xtrm0.99th.nc'))
    dg_valid_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_valid_Y_xtrm0.99th.nc')) 
    dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm0.99th.nc'))

In [5]:
# We need to pass zeros through the network to asses the baseline
dg_train_X_null = np.zeros(dg_train_X.shape)
dg_valid_X_null = np.zeros(dg_valid_X.shape)

In [6]:
# Compute weights for the weighted binary crossentropy
weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(pr_xtrm),
    y=pr_xtrm.flatten()
)

print('Weights for the weighted binary crossentropy:')
print(f'Classes: {np.unique(pr_xtrm)}, weights: {weights}')

# Create loss function for the extremes
xtrm_loss = weighted_binary_cross_entropy(
    weights={0: weights[0].astype('float32'), 1: weights[1].astype('float32')})

Weights for the weighted binary crossentropy:
Classes: [0 1], weights: [ 0.50508104 49.70253165]


In [7]:
# Define hyperparameters
BATCH_SIZE = 64
LR_METHOD = 'Constant'  # Cyclical, CosineDecay, Constant
                                            
# Default model options
opt_model = {'latent_dim': 128,
             'dropout_rate': 0.2}

# Default optimizer options
opt_optimizer = {'lr_method': 'Constant',
                 'lr': 0.0004,
                 'init_lr': 0.01}

models = {
          'UNET1': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 1, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET2': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 2, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET3': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 3, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET4': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 4, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET-att': {'model': 'Unet-attention', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET1-att': {'model': 'Unet-attention', 'run': True,
                       'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 1, 'unet_use_upsample': True},
                       'opt_optimizer': {'lr_method': 'Constant'}}
            }


In [8]:
##############LRP ###########
EPOCHS = 50
m_id = 'UNET4'
model = models[m_id]['model']
opt_model_i = models[m_id]['opt_model']
opt_optimizer_i = models[m_id]['opt_optimizer']

opt_model_new = opt_model.copy()
opt_model_new.update(opt_model_i)
opt_optimizer_new = opt_optimizer.copy()
opt_optimizer_new.update(opt_optimizer_i)

In [9]:
# Extremes
m = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=True,**opt_model_new)

# compile 
m.model.compile(loss=keras.losses.categorical_crossentropy, ## instead of CategoricalCrossentropy
                  optimizer='adam', ## lr instead of learning_rate
                  metrics=['categorical_accuracy'])

2022-12-15 16:13:28.301400: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 16:13:29.030485: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


In [10]:
# Train 
m.model.fit(dg_train_X_null, dg_train_Y_xtrm, validation_data=(dg_valid_X_null, dg_valid_Y_xtrm), 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0) 


2022-12-15 16:13:39.437892: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


In [11]:
m.model.save_weights('tmp/tmp_weights_DNN/UNET4_NULL_0.99th_trained_weights.h5')

In [14]:
# Not extremes
m_pr = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=False,**opt_model_new)

# compile 
m_pr.model.compile( loss='mse',  metrics=['mse'],  optimizer='adam')

2022-12-15 15:54:24.431420: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 15:54:25.160438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


In [15]:
m_pr.model.fit(dg_train_X_null, dg_train_Y, validation_data=(dg_valid_X_null, dg_valid_Y), 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0)

2022-12-15 15:54:35.594467: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


In [16]:
m_pr.model.save_weights('tmp/tmp_weights_DNN/UNET4_NULL_pr_trained_weights.h5')